In [ ]:

### Import libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
from sklearn.decomposition import PCA
import tqdm
from google.colab import files
np.random.seed(137)
%matplotlib inline

### Plot configuration

fig_width = 4.2 # width in inches
fig_height = 3.  # height in inches
fig_size =  [fig_width,fig_height]
plt.rcParams['figure.figsize'] = fig_size
plt.rcParams['figure.autolayout'] = True
plt.rcParams['font.size'] = 12
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

import os
print( os.getcwd() )


/content


# **##################### Preliminaries #####################**

To obtain the data you have two options:

# **Option 1**

You can load it 

```
import hdf5storage
p3_info = hdf5storage.loadmat('pset_p3.mat') 
```


In [ ]:
#@title Initialize and load data
# Load packages; 
import time
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Some more definitions for plotting
colors = np.array([[0.1254902 , 0.29019608, 0.52941176],
       [0.80784314, 0.36078431, 0.        ],
       [0.30588235, 0.60392157, 0.02352941],
       [0.64313725, 0.        , 0.        ]])
figsize = (8, 3)
# In order to load the data, do the following: 
# 1. Connect your google drive with the colab notebook via this part here:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


# Load the data file
try:
    import hdf5storage
except:
    !pip install hdf5storage
    import hdf5storage

# 2. Upload the dataset 'pset_p3.mat' to your google drive (top level; else
# change the data_dir below)
data_dir = "drive/MyDrive/pset_p3.mat"
p3_info = hdf5storage.loadmat(data_dir)

# Load variables into name space
# Number of neurons, g
N = p3_info["N"]
g = p3_info["g"]
# Activity before and after training
tData = p3_info["t"]
R0 = p3_info["R0"]
R = p3_info["R"]
# Training targets
target_t = p3_info["target_t"]
target_R = p3_info["target_activity"]
# Weight matrices
J0 = p3_info["J0"]
J = p3_info["J"]


# **Option 2**

In case you dont have the data, you can generate it by executing the cell below, but will take 5 mins 

In [ ]:
#@title You should only open (via double click) and run this cell if you couldn't load the simulated data!


#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####



#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### Generate  Target Bump

def RNN_target(N,dt,tData):
  dtData = 0.0641 
  # Generate time vecs
  t = np.arange(0, tData[-1], dt)

  # Generating the target activity (bump)
  xBump = np.zeros((N, len(tData)))
  sig = 0.0343 * N    # % scaled correctly in neuron space!!!

  # Generate Bump
  for i in range(N):
    xBump[i, :] = get_nonnormalized_gaussian(i+1, N * tData / tData[-1],sig)
  hBump = np.log((xBump + 0.01)/(1 - xBump + 0.01))     # current from rate
  return xBump,hBump

def get_nonnormalized_gaussian(x,mean,std):
      x=float(x)
      return np.exp(-(x-mean)**2/(2*std**2))


#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### Generate  Untrained Activity

def get_untrained_activity(N,g,dt,t,sig0,tau,xBump):


  # Generate simulated firing rate data from untrained RNN (for comparison)
  J0 = g * np.random.randn(N, N) / np.sqrt(N) # initial connectivity
  
  #Noise
  sigN = sig0 * np.sqrt(tau / dt)

  #Initialize
  R0 = np.zeros((N, len(t)))   # storing firing rates
  H0 = xBump[:, 0, np.newaxis] 

  # get aux input
  input=gen_aux_input(N,dt,t)
  
  for tt in range(1, len(t)):
      R0[:, tt, np.newaxis] = 1 / (1 + np.exp(-H0))
      noise = np.random.randn(N, 1)
      # JR =  J phi(x) + I + noise (compare with slide)
      JR = input[:, tt, np.newaxis] + sigN * noise + J0.dot(R0[:, tt]).reshape((N, 1))
      # Euler step for H i.e. dot(x) = -x + J phi(x) + I + noise (compare with slide)
      H0 = H0+ dt * (-H0 + JR) / tau
  return R0


#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### Training RNN

def gen_aux_input(N,dt,t):
  # input
  tauWN = 1
  ampWN = np.sqrt(tauWN/dt)
  ampIn = 1

  iWN = ampWN * np.random.randn(N, len(t))
  input = np.ones((N, len(t)))
  for tt in range(1, len(t)):
      input[:, tt] = iWN[:, tt] + (input[:, tt - 1] - iWN[:, tt]) * np.exp(- (dt / tauWN))
  input = ampIn * input
  return input



def train_RNN(N,g,dt,t,sig0,tau,xBump,hBump):

  print(" Implementation of the RLS algorithm running. Takes about ~5 minutes")


  # Initial connectivity
  J0 = g * np.random.randn(N, N) / np.sqrt(N) 

  # Generate std of noise
  sigN = sig0 * np.sqrt(tau / dt)

  # get aux input
  input=gen_aux_input(N,dt,t)

  # Learning parameters
  nRunTot = 125    # number of training iterations
  nFree = 5        # don't train for last nFree iterations
  nLearn = 50      # number of neurons being trained out of N
  P0 = 1.          # for initializing the RLS covariance matrix

  # Initialize

  permuted_neurons = np.random.permutation(N)
  cL = permuted_neurons[:nLearn]
  ncL = permuted_neurons[nLearn:]
  J = J0.copy() # Recurrence matrix 
  R = np.zeros((N, len(t)))   # storing firing rates
  JR = np.zeros((N, 1))
  PJ = P0 * np.eye(nLearn, nLearn)  # initial covariance

  # Train!

  for nRun in tqdm.tqdm((range(nRunTot))):
    H = xBump[:, 0, np.newaxis] 
    # print(nRun)
    tLearn = 0 # time-stamp resets to 0 everytime it reaches dtData (GCamP)
    iLearn = 1 # index of ?
    for tt in range(1, len(t)):
        tLearn = tLearn + dt
        R[:, tt, np.newaxis] = 1 / (1 + np.exp(-H))
        noise = np.random.randn(N, 1)
        # JR =  J phi(x) + I + noise (compare with slide)
        JR = input[:, tt, np.newaxis] + sigN * noise + J.dot(R[:, tt]).reshape((N, 1))
        # Euler step for H i.e. dot(x) = -x + J phi(x) + I + noise (compare with slide)
        H = H + dt * (-H + JR) / tau
        if tLearn >= dtData and nRun < (nRunTot - nFree):
          tLearn = 0
          err = JR[0:N, :] - hBump[0:N, iLearn, np.newaxis]
          iLearn = iLearn + 1
          r_slice = R[cL, tt].reshape(nLearn, 1)
          k = PJ.dot(r_slice)
          rPr = (r_slice).T.dot(k)[0, 0]
          c = 1.0 / (1.0 + rPr)
          PJ = PJ - c * (k.dot(k.T))
          J[0:N, cL.flatten()] = J[:, cL.reshape((nLearn))] - c * np.outer(err.flatten(), k.flatten())

  # np.savez('training_data.npz', J, J0, R)
  return J, J0, R

#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####


#### Define Network  parameters
g = 1.5
N = 500

# Simulation parameters
dt = 0.001
epochs = 165
tau = 0.01
sig0= 0.5
# Target  parameters

dtData = 0.0641 # GCamp time-scale
tData = np.arange(0, (epochs+1) * dtData, dtData) # time-indexes for GCamp Sim.

t = np.arange(0, tData[-1], dt) # time-indexes for RNN


#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### ####
#### Get target, untrained, and trained nets

xBump,hBump=RNN_target(N,dt,tData)

J, J0, R = train_RNN(N,g,dt,t,sig0,tau,xBump,hBump)

R0 = get_untrained_activity(N,g,dt,t,sig0,tau,xBump)




 Implementation of the RLS algorithm running. Takes about ~5 minutes


  1%|          | 1/125 [00:02<04:47,  2.32s/it]


KeyboardInterrupt: ignored

In [ ]:
np.savez('training_data.npz', J, J0, R,R0,t,tData,N,g)

# **##################### Problem 3 #####################**

The loaded data are simulation results from training a 500-neuron RNN to mimic target functions similar to calcium fluorescence signals collected from a population of active neurons in the prefrontal cortex of a mammalian nervous system.

 You should find 
 ```
N     # N neurons
g     # variance of the initial weights

tData # time vector 

xBump # Target activity

R0    # Rates of untrained network  (N x time)
R     # Rates of trained network  (N x time)

J0    # Initial random interaction matricex (NxN)
J     # Trained interaction matricex (NxN)

```
